# Använda Notebooks med elastic

Ett snabbt exempel på att använda notebooks med elastic med hjälp av Python.

Inspiration kommer från följande blogginlägg https://www.elastic.co/blog/analyzing-network-packets-with-wireshark-elasticsearch-and-kibana.

## Elastic

Starta elastic genom att skriva

    docker run --rm --name elasticsearch -v C:\presentations\notebooks-sv\elastic\elasticsearch.yml:/usr/share/elasticsearch/config/elasticsearch.yml:ro,Z --network presentation -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" elasticsearch:7.17.8

## Hämta pcap

Hämta och packa upp pcap från https://www.malware-traffic-analysis.net/2022/12/09/index.html. 

2022-12-09 (FRIDAY) - HTML SMUGGLING LEADS TO QAKBOT (QBOT), DISTRIBUTION/BOTNET TAG: AZD

In [ ]:
![[ ! -d pcap ]] && mkdir pcap

In [ ]:
![[ -d pcap ]] && [[ ! -e pcap/sample.zip ]] && curl -o pcap/sample.zip https://www.malware-traffic-analysis.net/2022/12/09/2022-12-09-azd-Qakbot-infection-traffic-carved-and-santized.pcap.zip
![[ -d pcap ]] && [[ -e pcap/sample.zip ]] && [[ ! -e pcap/sample.pcap ]] && cd pcap && 7z e -pinfected sample.zip && mv 2022* sample.pcap 

In [ ]:
# Extrahera data från pcap
![[ ! -e pcap/sample.json ]] && tshark -r pcap/sample.pcap -T ek | jq -c 'del(.index._type)' > pcap/sample.json
!head pcap/sample.json

In [ ]:
# Är elastic uppe (kan även köras i en terminal)
!curl -s elasticsearch:9200/_cluster/health?pretty

In [ ]:
# Ladda upp json till elastic
!cd pcap && curl -H "Content-Type: application/x-ndjson" -XPOST http://elasticsearch:9200/_bulk --data-binary @sample.json > /dev/null

## Länkar

- [Python Elasticsearch Client](https://elasticsearch-py.readthedocs.io/en/latest/) documentation
- [Elasticsearch DSL](https://elasticsearch-dsl.readthedocs.io/en/latest/index.html) documentation